In [3]:
import pandas as pd 
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab, Twitter
from konlpy.utils import pprint
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwords = Stopwords('./data/koreanStopwords.txt')

In [4]:
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'

In [123]:
pathlist = [path6, path7, path8, path9,]

In [130]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
def Check_Sentiment(pos, neg, x):
    y = list(chunker(x.split(), 2))
    y = list(filter(lambda x: len(x) !=1, y))
    for i,a in y:
        if i == 'NEGATIVE':neg.append(a.strip())
        else:
            pos.append(a)

In [132]:
pos = [] ; neg = []
pos_word = open('./data/positive_sentiment_word_from_news.csv','w')
neg_word = open('./data/negative_sentiment_word_from_news.csv','w')
pos_sentence = './data/positive_sentiment_sentence_from_news.csv'
neg_sentence = './data/negative_sentiment_sentence_from_news.csv'
for path in pathlist:
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python',encoding='utf-8')
        f = list(filter(lambda x: x[1] in ['감성분석','감성 분석','긍정문장','긍정 문장','부정 문장', '부정문장'], enumerate(df.columns.values)))
        redf = df.loc[:, df.columns[f[0][0]:]]
        redf.dropna(thresh=1, inplace = True)
        xc = redf['감성분석']
        xc = xc[xc.notnull()]
        xc = xc[xc != 'NEUTRAL']
        xc = xc.apply(lambda x: Check_Sentiment(pos, neg, x))
        xc2 = redf['긍정 문장']
        xc2 = xc2[xc2.notnull()]
        xc2 = xc2[xc2 != 'NEUTRAL']
        xc3 = redf['부정 문장']
        xc3 = xc3[xc3.notnull()]
        xc3 = xc3[xc3 != 'NEUTRAL']
        with open(pos_sentence,'a') as f:
            xc2.to_csv(f, header = False)
        with open(neg_sentence, 'a') as g:
            xc3.to_csv(g, header = False)

In [133]:
pos_word.write('\n'.join(pos))
pos_word.close()


In [134]:
neg_word.write('\n'.join(neg))
neg_word.close()

In [105]:
k

0                        [[NEGATIVE, 경계], [NEGATIVE, 극우]]
1       [[POSITIVE, 신뢰], [POSITIVE, 평화], [POSITIVE, 회복...
2                      [[POSITIVE, 발전], [POSITIVE, 증강하다]]
3                                        [[POSITIVE, 공헌]]
4       [[NEGATIVE, 걸림돌], [NEGATIVE, 좌절], [NEGATIVE, 의...
5       [[NEGATIVE, 진통], [NEGATIVE, 함정], [NEGATIVE, 모자...
6       [[NEGATIVE, 해임], [POSITIVE, 평화], [NEGATIVE, 못하...
7       [[POSITIVE, 생생하다], [POSITIVE, 쉽게해결되다], [NEGATI...
9       [[POSITIVE, 발전], [NEGATIVE, 불법], [NEGATIVE, 부당...
11      [[POSITIVE, 특수], [POSITIVE, 특수], [POSITIVE, 특수...
12      [[POSITIVE, 지원], [NEGATIVE, 까지], [NEGATIVE, 고자...
13      [[POSITIVE, 특수], [POSITIVE, 쉽다], [POSITIVE, 절약...
14      [[NEGATIVE, 미치], [NEGATIVE, 까지], [POSITIVE, 지원...
15                                       [[POSITIVE, 도움]]
16      [[POSITIVE, 탄탄], [POSITIVE, 관심], [POSITIVE, 완전...
17                       [[POSITIVE, 효과], [POSITIVE, 장점]]
19                                       [[POSITIVE, 비전]]
20      [[POSI

In [ ]:
fileis = open('./data/news/have_negative_positive/증강현실 관련 뉴스메타데이터(19900101-2016.9.30).csv')
while 1:
    line = fileis.readline()
    if not line:break
    print (line)
    


In [ ]:
pd.DataFrame(list(chunker(y2, 2)))

In [ ]:
excel_file2 = openpyxl.load_workbook('./data/NIADic.xlsx')

In [ ]:
excel_file2.sheetnames

In [ ]:
excel_sheet2 = excel_file2.get_sheet_by_name('NIADic')

In [ ]:
outDict = dict()
for row in excel_sheet2.rows:
    if row[0].row ==1:pass
    else:
        x = row[1].value.upper()
        if x in tagDict.keys():
            x2 = tagDict[x]
            if not x2 in outDict.keys():
                outDict[x2] = []
                outDict[x2].append(row[0].value)
            else:
                outDict[x2].append(row[0].value)

In [ ]:
twitter = Twitter()

In [ ]:
for idx in outDict:
    twitter.add_dictionary(outDict[idx], idx)

In [ ]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, 'newsDaum')

In [ ]:
rawdata = useCollection.find_one({'category':"뉴스"})
rawdata

In [ ]:
from ckonlpy.tag import Twitter
from konlpy.tag import Twitter as Original_Twitter

In [ ]:
tager_t = Twitter()

In [ ]:
tager_t.pos(rawdata['mainText'])

In [ ]:
def Extract_Keywords(text):
    twitter = list(filter(
            lambda x: x[1] =='Noun',
            Twitter().pos(text)))
    original_twitter = list(filter(
        lambda x: x[1] =='Noun',
        Original_Twitter().pos(text)))
    f11=Counter(twitter).most_common(n=10)
    f21=Counter(original_twitter).most_common(n=10)
    return set(map(lambda x: x[0][0], f11)) & set(map(lambda x: x[0][0], f21))

In [ ]:
for i in useCollection.find({'site':'daum'}):
    print (i['keywords'], Extract_Keywords(i['mainText']))

In [ ]:
f2 = list(filter(lambda x: x[1] =='Noun', Original_Twitter().pos(rawdata['mainText'])))

In [ ]:
set(f1) & set(f2)

In [ ]:
set(map(lambda x: x[0][0], f11)) & set(map(lambda x: x[0][0], f21))

In [ ]:
f11[0]